In [1]:
!pip install pandas
!pip install beautifulsoup4
!pip install twilio

from twilio.rest import Client
import pandas as pd
from datetime import datetime
import numpy as np
import bs4 as bs
import requests

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [2]:
wiki_link = 'https://en.wikipedia.org'
weblink = wiki_link + '/wiki/List_of_UFC_events'
response = requests.get(weblink)
soup = bs.BeautifulSoup(response.text, 'lxml')
table = soup.find_all('table', {'class':'wikitable'})[0]

In [5]:
ufc_events = pd.read_html(str(table),header=0)[0]
ufc_events.drop(columns='Ref.', inplace=True)
ufc_events.rename(columns={'Unnamed: 2':'Venue'}, inplace=True)

In [7]:
#add link to fight card

links=[]

for row in table.find_all('tr')[1:]:
    link = row.find_all('td')[0].contents[0]
    links.append(wiki_link + link.get('href'))
    
ufc_events['Fight Card Link'] = links

In [8]:
#fix date format

for index, row in ufc_events.iterrows():
    old_date = row['Date']
    new_date = datetime.strptime(old_date, '%b %d, %Y')
    row['Date'] = new_date.strftime('%d %b %Y')

In [9]:
ufc_events.tail(1)

,Event,Date,Venue,Location,Fight Card Link
25,UFC Fight Night: Gustafsson vs. Smith,01 Jun 2019,Ericsson Globe,"Stockholm, Sweden",https://en.wikipedia.org/wiki/UFC_Fight_Night:...


In [10]:
next_ufc_event = ufc_events.tail(1)
message_body = str(next_ufc_event.iloc[0,0]) + "\n" + str(next_ufc_event.iloc[0,3]) + "\n" + str(next_ufc_event.iloc[0,1]) + "\n" + str(next_ufc_event.iloc[0,4])
print(message_body)

UFC Fight Night: Gustafsson vs. Smith
Stockholm, Sweden
01 Jun 2019
https://en.wikipedia.org/wiki/UFC_Fight_Night:_Gustafsson_vs._Smith


In [11]:
account_sid = 'ACc2c5c19fd903a29de3492a7f1f110378'
auth_token = 'd8d3d91d148c07cfc5c4561b0286131d'
client = Client(account_sid, auth_token)

ed = 'whatsapp:+447771580615'
jay = 'whatsapp:+447749956110'

In [12]:
def send_whatsapp(recipient, message_body):
    message = client.messages.create(
                                  body= message_body,
                                  from_= 'whatsapp:+14155238886',
                                  to= recipient
                              )

    print(message.sid)

In [13]:
send_whatsapp(jay, message_body)

SM83f2c01d354246b78ff23638c94df789
